In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from IPython.display import display, Image
import pickle
from scipy import ndimage, misc
import cv2
import math
import matplotlib.pyplot as plt
# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

## Getting steering data for each frame

In [ ]:
base_dir = '/dev-box/datasets/dataset-20161027/Train/11'
interpolated = pd.read_csv(os.path.join(base_dir, 'interpolated.csv'))
print(interpolated.angle.max(), interpolated.angle.min(), interpolated.angle.count())

## Exploring the steering data

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.add_subplot(311, title='Steering wheel angle')
plt.plot(interpolated.timestamp, interpolated.angle)
fig.add_subplot(312, title='Speed')
plt.plot(interpolated.timestamp, interpolated.speed)
fig.add_subplot(313, title='Steering wheel torque')
plt.plot(interpolated.timestamp, interpolated.torque)

### Finding "stopped" intervals

Let's consider the vehicle is stopped when we have very low speed.

In [ ]:
def get_stopped(data):
    #return data.speed < .5 * (data.speed.mean() - data.speed.std()) # & (abs(ts_center.steering_wheel_torque) < .1)
    return data.speed < 3.

stopped = get_stopped(interpolated)

fig = plt.figure(figsize=(10, 3))

#fig.add_subplot(311)
plt.plot(interpolated.timestamp, stopped)
ax = plt.axis()
plt.axis((ax[0], ax[1], ax[2], 1.2))

stopped.head()

### Finding sharp turns

Sharp turns are characterized by high_angles | (high_torque & low_speed).

In [ ]:
# some statistics
steering_wheel_angle_mean = steering.steering_wheel_angle.mean()
steering_wheel_angle_std = steering.steering_wheel_angle.std()
steering_wheel_torque_mean = steering.steering_wheel_torque.mean()
steering_wheel_torque_std = steering.steering_wheel_torque.std()
speed_mean = steering.speed.mean()
speed_std = steering.speed.std()
print('Steering angle mean: {0:.3f} +/- {1:.3f}'.format(steering_wheel_angle_mean, steering_wheel_angle_std))
print('Steering torque mean: {0:.3f} +/- {1:.3f}'.format(steering_wheel_torque_mean, steering_wheel_torque_std))
print('Speed mean: {0:.3f} +/- {1:.3f}'.format(speed_mean, speed_std))

In [ ]:
def get_sharp_turns(data):
    high_angles = abs(data.angle) > 0.5
    return high_angles

sharp = get_sharp_turns(interpolated)
#sharp_left = get_sharp_turns(ts_left)
#sharp_right = get_sharp_turns(ts_right)

fig = plt.figure(figsize=(10, 3))
#fig.add_subplot(311)
plt.plot(interpolated.index, sharp)
ax = plt.axis()
plt.axis((ax[0], ax[1], ax[2], 1.2))

'left' in str(interpolated.frame_id.values[0])

### Frames to exclude

Since we are only interested in following lanes, we exclude sharp turns and stopped frames.

In [ ]:
exclude_center = stopped_center | sharp_center
exclude_right = stopped_right | sharp_right
exclude_left = stopped_left | sharp_left

fig = plt.figure(figsize=(10, 8))
fig.add_subplot(311)
plt.plot(ts_center.index, exclude_center)
ax = plt.axis()
plt.axis((ax[0], ax[1], ax[2], 1.2))

fig.add_subplot(312)
plt.plot(ts_left.index, exclude_left)
ax = plt.axis()
plt.axis((ax[0], ax[1], ax[2], 1.2))

fig.add_subplot(313)
plt.plot(ts_right.index, exclude_right)
ax = plt.axis()
plt.axis((ax[0], ax[1], ax[2], 1.2))

### Filtered frames

In [ ]:
fig = plt.figure(figsize=(10, 14))
print(ts_center.shape, ts_center[include_center].shape)

filtered_center = ts_center[~exclude_center]
filtered_left = ts_left[~exclude_left]
filtered_right = ts_right[~exclude_right]
#filtered_center.reset_index(inplace=True)

fig.add_subplot(411, title='Original steering wheel angle')
plt.plot(ts_center.index, ts_center.steering_wheel_angle)
fig.add_subplot(412, title='Steering wheel angle')
plt.plot(range(filtered_center.shape[0]), filtered_center.steering_wheel_angle)
fig.add_subplot(413, title='Speed')
plt.plot(range(filtered_center.shape[0]), filtered_center.speed)
fig.add_subplot(414, title='Steering wheel torque')
plt.plot(range(filtered_center.shape[0]), filtered_center.steering_wheel_torque)

# Image processing


## Show random images from cameras

Assuming images are already extracted from rosbag file.

In [ ]:
image_folders = ['data/left_camera', 'data/center_camera', 'data/right_camera']

fig = plt.figure(figsize=(12, 10))
choice = np.random.choice(os.listdir(image_folders[i]))

for i in range(len(image_folders)):    
    img = cv2.imread(os.path.join(image_folders[i], choice))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig.add_subplot(1, 3, i+1)
    plt.imshow(img)
    #display(Image(os.path.join(image_folders[i], np.random.choice(os.listdir(image_folders[i])))))

## Dataset preparation

Steps:
* Read image
* Convert colorspace to YUV
* Generate new transformed images and labels
* Resize images
* Serialize to TFRecord

In [ ]:
import numpy as np
import tensorflow as tf
import augmentation as aug
from tqdm import tnrange, tqdm_notebook

def create_example(image, data, colorspace):
    
    feature = {}
    
    feature['steering_angle'] = tf.train.Feature(
            float_list=tf.train.FloatList(value=[data.get('steering_wheel_angle')]))
    
    feature['timestamp'] = tf.train.Feature(
            float_list=tf.train.FloatList(value=[data.get('timestamp')]))
    
    feature['shift'] = tf.train.Feature(
            float_list=tf.train.FloatList(value=[data.get('shift')]))
        
    if data.get('steering_wheel_torque'):
        feature['steering_torque'] = tf.train.Feature(
                float_list=tf.train.FloatList(value=[data.get('steering_wheel_torque')]))
        
    if data.get('speed'):
        feature['speed'] = tf.train.Feature(
                float_list=tf.train.FloatList(value=[data.get('speed')]))
        
    feature['image/height'] = tf.train.Feature(
            int64_list=tf.train.Int64List(value=[image.shape[0]]))
    
    feature['image/width'] = tf.train.Feature(
            int64_list=tf.train.Int64List(value=[image.shape[1]]))
    
    feature['image/channels'] = tf.train.Feature(
            int64_list=tf.train.Int64List(value=[image.shape[2]]))
    
    feature['image/colorspace'] = tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[colorspace]))
    
    feature['image'] = tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[image.tostring()]))
    
    # construct the Example proto object
    example = tf.train.Example(
        # Example contains a Features proto object
        features=tf.train.Features(
          # Features contains a map of string to Feature proto objects
          feature=feature
    ))
    
    return example

def create_transformed_example(image, data, colorspace, rotation=None, shift=None, initial_shift=0, 
                               width=None, height=None):
    
    '''transformed, steering_wheel_angle, r, s = aug.steer_back_distortion(image, 
                                                                        data.steering_wheel_angle, 
                                                                        data.speed,
                                                                        rotation=rotation, shift=shift,
                                                                        initial_shift=initial_shift
                                                                       )'''
    transformed_data = {
        'steering_wheel_angle': data.steering_wheel_angle,
        'speed': data.speed,
        'timestamp': data.timestamp,
        'shift': initial_shift
    }

    #if width and height:
    #    transformed = cv2.resize(transformed,(width, height))

    return create_example(transformed, transformed_data, image_colorspace)


image_width_small = 200
image_height_small = 90
image_channels = 3
image_colorspace = b'YUV'

filtered_idx = [filtered_left.index, filtered_center.index, filtered_right.index]
steering_files = ['data/dataset-20160929/left_steering.csv', 
                  'data/dataset-20160929/center_steering.csv', 
                  'data/dataset-20160929/right_steering.csv']
image_folders = ['data/dataset-20160929/left_camera', 
                 'data/dataset-20160929/center_camera', 
                 'data/dataset-20160929/right_camera']
initial_shifts = [-.5, 0, .5]
#augmentation_factor = 1
images_per_file = 2000

for i in tnrange(len(image_folders)):
    
    folder = image_folders[i]
    
    image_files = os.listdir(folder)
    idxs = np.array(filtered_idx[i])
    #idxs = np.random.choice(len(image_files), 20)
    
    steering = pd.read_csv(steering_files[i])
    
    # iterate over each example
    # wrap with tqdm for a progress bar
    file_count = math.ceil(len(idxs) / images_per_file)
    
    for j in tnrange(file_count):
        
        filename = '{}@{}x{}_{}.tfrecords'.format(folder, image_height_small, image_width_small, j)
        writer = tf.python_io.TFRecordWriter(filename)
        
        images_in_file = images_per_file
        
        if (len(idxs) - j * images_per_file) < images_per_file:
            images_in_file = len(idxs) % images_per_file
        
        for k in tnrange(images_in_file):
        
            example_idx = idxs[j * images_per_file + k]
            
            try:
                img = cv2.imread(os.path.join(folder, image_files[example_idx]))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
            except:
                print('Could not read:', os.path.join(folder, image_files[example_idx]), '- it\'s ok, skipping.')
                continue
            
            data = steering.iloc[example_idx]

            example = create_transformed_example(img, data, image_colorspace, initial_shift=initial_shifts[i],
                                                 rotation=0, shift=0)

            # use the proto object to serialize the example to a string
            serialized = example.SerializeToString()

            # write the serialized object to disk
            writer.write(serialized)
            
            '''for k in range(augmentation_factor-1):
                
                example = create_transformed_example(img, data, image_colorspace, initial_shift=initial_shifts[i], 
                               width=image_width_small, height=image_height_small)
                serialized = example.SerializeToString()

                writer.write(serialized)'''
                
        writer.close()
    

## Check images

In [ ]:
i = 0
data_dir = 'data/eval'
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.tfrecords')]
print(files)
for serialized_example in tf.python_io.tf_record_iterator(files[0]):
    
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    
    # traverse the Example format to get data
    height = example.features.feature['image/height'].int64_list.value[0]
    width = example.features.feature['image/width'].int64_list.value[0]
    channels = example.features.feature['image/channels'].int64_list.value[0]
    image = example.features.feature['image'].bytes_list.value[0]
    image = np.fromstring(image, np.uint8)
    image = np.asarray(image)
    image = image.reshape(height, width, channels)
    #steering = example.features.feature['steering_angle'].float_list.value[0]
    
    if i < 10:
        fig = plt.figure()
        plt.imshow(image)
        #print(steering)
    i = i + 1